In [1]:
#Set up von Spark
import findspark
findspark.init()
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("BigData").getOrCreate()
sc = spark.sparkContext
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/23 21:17:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#Festlegen der Speicherorte der Daten
#Daniel
folder_Temperature = '/Users/danielwentsch/Desktop/BigData/Data/02_air_temperature/'
folder_Solar = '/Users/danielwentsch/Desktop/BigData/Data/04_solar/'
#Mario
#folder_Temperature = 
#folder_Solar = 


In [3]:
#Einlesen der jeweiligen Daten in einen DataFrame

#Lufttemperatur
#Daniel
df_Temperatur = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Temperature+'*.txt')
#Mario
#df_Temperatur = spark.read.options(delimiter=";", inferSchema='True', header=True).csv(folder_Temperature)
#Sonnenscheindauer
#Daniel
df_Solar = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(folder_Solar+'*.txt')
#Mario
#df_Solar = spark.read.options(delimiter=";", inferSchema='True', header=True).csv(folder_Solar)


Datenstruktur verstehen 
Daten bearbeiten und in einen Finalen DF schreiben 
Infos: Zu jeder Station ID sollen zu jedem Timestmap die jeweiligen Daten zusammengefasst werden


In [4]:
df_Temperatur.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)
 |-- eor: string (nullable = true)




Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel) TT_10 (Luftemperatur in 2m Höhe) und TM5_10 (Luftemperatur in 2cm Höhe) und PP_10(Luftdruck)
Ändern der Datentypen von TT_10, TM5_10 und PP_10 in float 

In [5]:
df_Solar.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |--   QN: string (nullable = true)
 |-- DS_10: string (nullable = true)
 |-- GS_10: string (nullable = true)
 |-- SD_10: string (nullable = true)
 |-- LS_10: string (nullable = true)
 |-- eor: string (nullable = true)



Aus diesem Datensatz benötigen wir die Spalten StationsID und MESS_Datum (Zeitstempel), SD_10 (10 min-Summe der Sonnenscheindauer)
Ändern der Datentypen von SD_10 in float

In [6]:
#Bearbeiten der Struktur in Geünschtes Format 
#Für df_temperatur ID, Datum, TT_10, TM5_10 und PP_10 und umbennen 
df_Temperatur  = df_Temperatur.withColumn("Jahr", df_Temperatur.MESS_DATUM.substr(1,4))
df_Temperatur  = df_Temperatur.withColumn("Monat", df_Temperatur.MESS_DATUM.substr(5,2))
df_Temperatur  = df_Temperatur.withColumn("Tag", df_Temperatur.MESS_DATUM.substr(7,2))
df_Temperatur  = df_Temperatur.withColumn("Stunde", df_Temperatur.MESS_DATUM.substr(9,2))
df_Temperatur  = df_Temperatur.withColumn("Minute", df_Temperatur.MESS_DATUM.substr(11,2))
#Datentypen bearbeiten 
df_Temperatur = df_Temperatur.withColumn('STATIONS_ID',df_Temperatur.STATIONS_ID.cast(IntegerType())).withColumn('TM5_10',df_Temperatur.TM5_10.cast(FloatType())).withColumn('PP_10',df_Temperatur.PP_10.cast(FloatType())).withColumn('TT_10',df_Temperatur.TT_10.cast(FloatType())).withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))
#Daten droppen
df_Temperatur = df_Temperatur.drop('  QN','RF_10','TD_10','eor')
#Spalten umbennen
df_Temperatur = df_Temperatur.withColumnRenamed('TT_10','Temperatur_2m').withColumnRenamed('TM5_10','Temperatur_5cm').withColumnRenamed('PP_10','Luftdruck')
df_Temperatur.show(3,False)

+-----------+-------------------+---------+-------------+--------------+----+-----+---+------+------+
|STATIONS_ID|MESS_DATUM         |Luftdruck|Temperatur_2m|Temperatur_5cm|Jahr|Monat|Tag|Stunde|Minute|
+-----------+-------------------+---------+-------------+--------------+----+-----+---+------+------+
|2171       |2010-01-01 00:00:00|960.8    |-1.3         |-1.3          |2010|01   |01 |00    |00    |
|2171       |2010-01-01 00:10:00|960.8    |-1.4         |-1.3          |2010|01   |01 |00    |10    |
|2171       |2010-01-01 00:20:00|960.7    |-1.4         |-1.3          |2010|01   |01 |00    |20    |
+-----------+-------------------+---------+-------------+--------------+----+-----+---+------+------+
only showing top 3 rows



In [7]:
df_Solar  = df_Solar.withColumn("Jahr", df_Solar.MESS_DATUM.substr(1,4))
df_Solar  = df_Solar.withColumn("Monat", df_Solar.MESS_DATUM.substr(5,2))
df_Solar  = df_Solar.withColumn("Tag", df_Solar.MESS_DATUM.substr(7,2))
df_Solar  = df_Solar.withColumn("Stunde", df_Solar.MESS_DATUM.substr(9,2))
df_Solar  = df_Solar.withColumn("Minute", df_Solar.MESS_DATUM.substr(11,2))
#Datentypen bearbeiten
df_Solar = df_Solar.withColumn('SD_10',df_Solar.SD_10.cast(FloatType())).withColumn("MESS_DATUM",to_timestamp("MESS_DATUM", "yyyyMMddHHmm"))
#Daten droppen 
df_Solar = df_Solar.drop('  QN','DS_10','GS_10','LS_10','eor')
#Spalte numbennen
df_Solar = df_Solar.withColumnRenamed('SD_10','Sonnenscheindauer')
df_Solar.show(3,False)

+-----------+-------------------+-----------------+----+-----+---+------+------+
|STATIONS_ID|MESS_DATUM         |Sonnenscheindauer|Jahr|Monat|Tag|Stunde|Minute|
+-----------+-------------------+-----------------+----+-----+---+------+------+
|         73|2010-01-01 00:00:00|0.0              |2010|01   |01 |00    |00    |
|         73|2010-01-01 00:10:00|0.0              |2010|01   |01 |00    |10    |
|         73|2010-01-01 00:20:00|0.0              |2010|01   |01 |00    |20    |
+-----------+-------------------+-----------------+----+-----+---+------+------+
only showing top 3 rows



In [8]:
#Join der DF
df_Final = df_Temperatur.join(df_Solar, (df_Temperatur.STATIONS_ID == df_Solar.STATIONS_ID) & (df_Temperatur.MESS_DATUM == df_Solar.MESS_DATUM)).select(df_Temperatur['*'],df_Solar.Sonnenscheindauer)
df_Final = df_Final.withColumn('Monat',df_Temperatur.Monat.cast(IntegerType()))
df_Final = df_Final.withColumn('Jahr',df_Temperatur.Jahr.cast(IntegerType()))
df_Final = df_Final.withColumn('Tag',df_Temperatur.Tag.cast(IntegerType()))
df_Final = df_Final.withColumn('Stunde',df_Temperatur.Stunde.cast(IntegerType()))
df_Final = df_Final.withColumn('Minute',df_Temperatur.Minute.cast(IntegerType()))
df_Final.printSchema()


root
 |-- STATIONS_ID: integer (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- Luftdruck: float (nullable = true)
 |-- Temperatur_2m: float (nullable = true)
 |-- Temperatur_5cm: float (nullable = true)
 |-- Jahr: integer (nullable = true)
 |-- Monat: integer (nullable = true)
 |-- Tag: integer (nullable = true)
 |-- Stunde: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- Sonnenscheindauer: float (nullable = true)



Wieso hat es hier weniger Datensätze als beide einzeln?

In [9]:
#Löschen der Spalten, in welchen Werte fehlen (-999)
df_Final = df_Final.filter((df_Final.Sonnenscheindauer != -999.0))
df_Final = df_Final.filter((df_Final.Luftdruck != -999.0))
df_Final = df_Final.filter((df_Final.Temperatur_2m != -999.0))
df_Final = df_Final.filter((df_Final.Temperatur_5cm != -999.0))

In [10]:
#DF aus dem df_Final erzeugen welcher alle Kmbinationen der StationsIDs und Jahre enthält
df_Kombinationen =  df_Final.withColumn("ID",concat_ws('','Jahr','STATIONS_ID'))
df_Kombinationen = df_Kombinationen.select("ID",'STATIONS_ID','Jahr')
df_Kombinationen = df_Kombinationen.dropDuplicates(["ID"]) 

In einem nächsen Schritt soll der Spruch "Ist der Januar hell und weiß, wird der Sommer Sicher heiß" geprüft werden
Hierfür müssen die Daten Pro Jahr und StationsID betrachtet werden. Zusätzlich sind pro Jahr nur die Temperaturen von 21. Juni bis 23. September und die Sonnenscheindauer im Januar wichtig 

In [11]:
#Zu DF df_Kombinationen eine neue Spalte hinzufügen, welche die Summe an Sonnenstunden pro StationsID und Jahr enthält 
df_Final_Solar_Grouped = df_Final.filter((df_Final.Monat == 12)).groupBy('STATIONS_ID','Jahr','Monat').mean('Sonnenscheindauer')
df_Final_Solar_Grouped = df_Final_Solar_Grouped.withColumnRenamed('avg(Sonnenscheindauer)','Sonnenscheindauer')
df_Kombinationen = df_Kombinationen.join(df_Final_Solar_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Solar_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Solar_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Solar_Grouped.Sonnenscheindauer)
df_Kombinationen = df_Kombinationen.sort('Jahr')
df_Kombinationen = df_Kombinationen.sort('STATIONS_ID')

In [12]:
df_Final_Temperatur_Grouped = df_Final.filter((df_Final.Monat == 7)|(df_Final.Monat == 8)).groupBy('STATIONS_ID','Jahr').mean('Temperatur_2m')
df_Final_Temperatur_Grouped = df_Final_Temperatur_Grouped.withColumnRenamed('avg(Temperatur_2m)','Temperatur_2m')
df_Kombinationen = df_Kombinationen.join(df_Final_Temperatur_Grouped,(df_Kombinationen.STATIONS_ID == df_Final_Temperatur_Grouped.STATIONS_ID)&(df_Kombinationen.Jahr == df_Final_Temperatur_Grouped.Jahr )).select(df_Kombinationen['*'],df_Final_Temperatur_Grouped.Temperatur_2m)
df_Kombinationen = df_Kombinationen.sort('Jahr')
df_Kombinationen = df_Kombinationen.sort('STATIONS_ID')

In [17]:
df_Kombinationen.show(45)

22/12/23 21:38:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
22/12/23 21:38:51 WARN RowBasedKeyValueBatch: Calling spill() on

+--------+-----------+----+--------------------+------------------+
|      ID|STATIONS_ID|Jahr|   Sonnenscheindauer|     Temperatur_2m|
+--------+-----------+----+--------------------+------------------+
|   19953|          3|1995| 0.01035866405079181| 20.51484892505696|
|   20003|          3|2000| 0.01490221998334118|16.634827434936483|
|   20033|          3|2003| 0.01812051935204869|20.168967065591062|
|   20083|          3|2008| 0.01682294288073879| 17.70253164877453|
|   20073|          3|2007|0.014033153854263779|17.097478986841622|
|   20103|          3|2010|0.005202956924108141| 19.04007616842092|
|   19993|          3|1999|0.005930076204223624|18.812432798433473|
|   19983|          3|1998|0.008539376322774307|16.921142600754923|
|   20043|          3|2004|0.018219085616363366|18.134386201058664|
|   19933|          3|1993|0.003029689622194...|16.413130403221103|
|   20053|          3|2005|0.008264112773628431|17.289135304189497|
|   19943|          3|1994| 0.01617198036904152|

In [ ]:
#Correlation zwischen Sonnenschein und Durschsnittstemperatur
corr_Spruch1 = df_Kombinationen.stat.corr('Sonnenscheindauer','Temperatur_2m')